# Lab 5 - Solidity Types


## 1. Setup

### Step 1: Copy content of lab-4 into lab-5

We will make use of the config, .env and packages installed from previous lab.


In [ ]:
cp -rp ~/github/suss/fin579/lab-4/. ~/github/suss/fin579/lab-5

## 2. Lab 5a: Uint256 is cheaper than UIn8

### Step 1.  Create the contract testUInt.sol in `contracts`.

    ```sol
    pragma solidity 0.8.8;

    contract TestUInt256 {
        uint256 public UInt256;
    }
    contract TestUInt8 {
        uint8 public UInt8;
    }
    ```

### Step 2.  Create the test testUInt.js in `test`.

### Step 3.  Test deploy TestUInt256 and estimate the gas fee.

    ```js
    it("Test deploy UInt256", async () => {
        const factory = await ethers.getContractFactory("TestUInt256");
        const txn = await factory.getDeployTransaction();
        const gas = await ethers.provider.estimateGas(txn);
        console.log(gas);
        // BigNumber { value: "92079" }
    });
    ```

### Step 4.  Test deploy UInt8 and estimate the gas fee.

    ```js
    it("Test deploy UInt8", async () => {
        const factory = await ethers.getContractFactory("TestUInt8");
        const txn = await factory.getDeployTransaction();
        const gas = await ethers.provider.estimateGas(txn);
        console.log("UInt8:", gas);
        // BigNumber { value: "95097" }
    });
    ```

In [ ]:
hh test test/testUInt.js

### Step 5.  Why does a contract with 8-bit integer state variable cost more than deploying a contract with 256-bit integer state variable?

## 3. Lab5b: Little-Endian vs Big-Endian

### Step 1. Create a contract `TestBytes32.sol` in `contracts`.

    ```sol
    pragma solidity 0.8.8;

    contract TestBytes32 {
        bytes32 public Bytes32 = '0x12345';
        function setBytes32(bytes32 b) public
        {
            Bytes32 = b;
        }

        function setUInt256(uint256 b) public
        {
            Bytes32 = bytes32(b);
        }

        function setString(string memory b) public
        {
            Bytes32 = bytes32(bytes(b));
        }
    }
    ```

### Step 2. Create a test `testBytes32.js` in `test`.

    ```js
    it("Test Bytes32", async () => {
        const factory = await ethers.getContractFactory("TestBytes32");
        const contract = await factory.deploy();

        // Set Hexastring
        await contract.setBytes32(ethers.utils.zeroPad("0x11", 32));
        let bytes32 = await contract.Bytes32();
        console.log("Bytes32: setBytes32", bytes32);
        // output: 0x0000000000000000000000000000000000000000000000000000000000000011

        // Set Number
        await contract.setUInt256(1);
        bytes32 = await contract.Bytes32();
        console.log("Bytes32: setUInt256", bytes32);
        // output: 0x0000000000000000000000000000000000000000000000000000000000000001

        // Set String
        await contract.setString("1");
        bytes32 = await contract.Bytes32();
        console.log("Bytes32: setString", bytes32);
        // output: 0x3100000000000000000000000000000000000000000000000000000000000000
    });
    ```


In [ ]:
hh test test/testBytes32.js

## 4. Lab 5c: Mapping

1. Create a contract `TestMapping.sol` in `contracts`.

    ```sol
    pragma solidity 0.8.8;

    contract TestMapping {
        address[] public _addresses;
        mapping(address=>uint256) public _addressToTokenId;

        function addAddress(address owner) private {
            _addresses.push(owner);
        }

        function setMapping(address owner, uint256 tokenId) public {
            addAddress(owner);
            _addressToTokenId[owner]=tokenId;
        }
    }
    ```

2. Create test `testMapping.sol` in `test`.

    ```js
    it("Test add multiple mappings", async () => {
        const factory = await ethers.getContractFactory("TestMapping");
        const mapping = await factory.deploy();
        const accounts = await ethers.getSigners();
        const keys = [...Array(10).keys()];

        // Create
        for (const key in keys) {
            await mapping.setMapping(accounts[key].address, key);
        }

        // List
        for (const key in keys) {
            const address = await mapping._addresses(key);
            const tokenId = await mapping._addressToTokenId(address);
            console.log(tokenId);
        }
    });
    ```

3. As an exercise, extend the TestMapping contract on your own to perform a check for duplicate on the address before adding the address to the \_addresses array.

In [ ]:
hh test test/testMapping.js

## 5. Lab 5d: Bytes and Strings

1. Create a contract `TestStringBytes.sol` in `contracts`.

    ```sol
    contract TestStringBytes {

        bytes public _bytes;
        string public _string;

        function convertStringtoBytes(string memory s) public
        {
            // cast string as bytes
            _bytes = bytes(s);
        }
        function concatenateString(string memory a, string memory b) public
        {
            // convert to bytes, convert to bytes then cast as string.
            _string = string(abi.encodePacked(a,b));
        }

    }
    ```

2. Create the test `testStringBytes.js`.

    ```js
    it("Convert string to bytes", async () => {
        const factory = await ethers.getContractFactory("TestStringBytes");
        const sb = await factory.deploy();
        await sb.convertStringtoBytes("Hello world");
        const bytes = await sb._bytes();
        console.log(ethers.utils.toUtf8String(bytes));
    });

    it("Concatenate strings", async () => {
        const factory = await ethers.getContractFactory("TestStringBytes");
        const sb = await factory.deploy();
        await sb.concatenateString("Hello", " world");
        const s = await sb._string();
        console.log(s);
    });
    ```

In [ ]:
hh test test/testStringBytes.js